In [1]:
import os
import numpy as np
import pandas as pd
import glob

# !pip install tables

In [9]:
source_folder = '/Users/justynrodrigues/Documents/nfl/data/pbp/csv/'
all_files = glob.glob('/Users/justynrodrigues/Documents/nfl/data/pbp/csv/*.csv.gz')

df = pd.concat((pd.read_csv(__, low_memory=False, index_col=False) for __ in all_files))

In [8]:
#df.filter(regex='_type').columns
#df = df.query("1 <= down <= 4")

In [10]:
# I am a coward so I am making a copy df just "in case"
df1 = df.copy()
df1 = df1.rename(columns={'time': 'play_time'}).query("1 <= down <= 4")

df1 = df1.assign(
    game_date = pd.to_datetime(df1.game_date),
    season_code = 'R' + df1.season.astype(str),
    defteam_type = np.where(df1.posteam_type == 'away', df1.defteam, df1.posteam),
    play_id2 = np.arange(len(df1)),
    fourth_down = np.where(df1.down == 4, 1, 0),
    fourth_down_attempt = np.where((df1.play_type.isin(['pass', 'run', 'qb_kneel', 'qb_spike'])) & (df1.down  == 4), 1, 0)
)

df1 = df1[['game_date', 'week', 'season', 'season_type', 'season_code', 'game_id', 'play_id', 'play_id2', 'qtr', 'drive', 'play_time', 'home_team', 'away_team', 'posteam',
'posteam_type', 'defteam_type', 'play_type', 'down', 'ydstogo', 'yardline_100', 'field_goal_attempt', 'fourth_down', 'fourth_down_attempt', 
'fourth_down_converted', 'fourth_down_failed', 'penalty']]

df1 = df1.dropna().reset_index()

# fourth_down_conversions = df.is_fourth_down_converted.astype(int)
# fourth_down_attempts = df.is_fourth_down_attempt.astype(int)

In [13]:
df1.to_csv('data/pbp.csv', index=False)

In [2]:
df1 = pd.read_csv('data/pbp.csv')

In [11]:
def summarize(df, grouping, cols):
    df = df.groupby(grouping)[cols]
    return df.sum().reset_index()

def add_pct(df):
    df['fourth_down_attempt_pct'] = df['fourth_down_attempt'] / df['fourth_down']
    df['fourth_down_conversion_pct'] = df['fourth_down_converted'] / df['fourth_down_attempt']
    return df

metric_cols = ['fourth_down', 'fourth_down_attempt', 'fourth_down_converted']

In [12]:
df_summary = add_pct(pd.DataFrame(df1[metric_cols].sum()).T)
df_summary

,fourth_down,fourth_down_attempt,fourth_down_converted,fourth_down_attempt_pct,fourth_down_conversion_pct
0,90926.0,11721.0,5790.0,0.128907,0.493985


In [7]:
df1.value_counts('fourth_down_attempt')

fourth_down_attempt
0    872020
dtype: int64